In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
import numpy as np
import torch.nn as nn
import string
from torch.utils.data import DataLoader, random_split
import copy
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import torch.nn.functional as F
from sklearn.metrics import classification_report

### Dataset Class

In [17]:
class EntityDataset(Dataset):

    def __init__(self, data_dir):
        """Initialize the attributes of the object of the class."""
        
        # data directory
        self.data_dir = data_dir
        
        # load text dataset  
        self.sentences = self._read_data(data_dir, 'sentences')

        # load text dataset  
        self.labels = self._read_data(data_dir, 'labels')  
        
    
       
        

    def __len__(self):
        """Return the size of the dataset."""
        return len(self.sentences)

    def __getitem__(self, index):
        """Return a data sample for a given index, along with the lable of the corresponding tweet"""
        
        
        # - get the data sample corresponding to 'index' (use the list 'self.image_path_list')
        data_sample = self.sentences[index]
        label = self.labels[index]
        
        features = {'sents': data_sample, 'labels': label}
        
        
        return features

 
    
    def _label_map(self,label,class_num=3):
        
        """ convert to labels to one hot vectors"""
        
        one_hot = torch.zeros(class_num, dtype=torch.int32)
        idx = self.label_to_index[label.upper()]
        if idx!=-1:
            one_hot[idx] = 1
        
        return one_hot
            
            
    
    def _read_data(self, path, pattern):
        
        """ read txt file and return as list of strings"""
        
        path = f'{path}/{pattern}.txt'
        with open(path, 'r') as file:
            data = [line.strip() for line in file]

        
        return data
         

### Create Dataloaders

In [18]:

dataset_test = EntityDataset('../Dataset/test')


In [19]:
batch_size = 1

test_dataloader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


In [22]:
# print an example batch

batch_example = next(iter(test_dataloader))
tweet_batch_example = batch_example['sents']
labels_batch_example = batch_example['labels']

print(tweet_batch_example[0].split())
print(labels_batch_example[0].split())

['Recent', 'work', 'in', 'natural', 'language', 'generation', 'has', 'begun', 'to', 'take', 'linguistic', 'variation', 'into', 'account', 'developing', 'algorithms', 'that', 'are', 'capable', 'of', 'modifying', 'the', 'system', "'s", 'linguistic', 'style', 'based', 'either', 'on', 'the', 'user', "'s", 'linguistic', 'style', 'or', 'other', 'factors', 'such', 'as', 'personality', 'or', 'politeness']
['O', 'O', 'O', 'B', 'I', 'I', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'O', 'O', 'B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [34]:
true = []
preds = []
for batch in iter(test_dataloader):
    for labels in batch['labels']:
        labels = labels.replace('0','O')
        true.extend(labels.upper().split())
        preds.extend(['O' for i in range(len(labels.split()))])
        
        
len(true)
len(preds)

2468

In [35]:
print(classification_report(true,preds))

              precision    recall  f1-score   support

           B       0.00      0.00      0.00       328
           I       0.00      0.00      0.00       339
           O       0.73      1.00      0.84      1801

    accuracy                           0.73      2468
   macro avg       0.24      0.33      0.28      2468
weighted avg       0.53      0.73      0.62      2468



/home/hossain/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hossain/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hossain/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
